# NIfTI 3D Volume Resampling - intoductory Guide

This notebook demonstrates how to resample MRI volumes with proper handling of:
- Rotated affine matrices
- Multi-axis resampling (x, y, z independently)
- Different interpolation methods
- Quality comparison and visualization

In [1]:
# Install required packages if needed
# !pip install nibabel numpy scipy matplotlib --break-system-packages

In [2]:
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom as scipy_zoom
import matplotlib.pyplot as plt
from pathlib import Path

## Understanding Spacing from Affine Matrix

**Critical**: For rotated volumes, spacing is NOT the diagonal of the affine matrix!

In [3]:
def get_spacing_from_affine(affine):
    """
    Extract voxel spacing from affine matrix.
    Works correctly even with rotated affine matrices.
    
    Parameters:
    -----------
    affine : ndarray
        4x4 affine matrix
    
    Returns:
    --------
    ndarray
        (x, y, z) spacing in mm
    """
    # Spacing is the magnitude of each column vector
    return np.sqrt(np.sum(affine[:3, :3] ** 2, axis=0))

# Example with rotated affine
rotated_affine = np.array([
    [-0.01045284,  0.0,        0.09945219, -2.93459034],
    [ 0.0,        -0.1,        0.0,         8.0],
    [ 0.09945219,  0.0,        0.01045284, -5.08460236],
    [ 0.0,         0.0,        0.0,         1.0]
])

print("Example affine matrix with rotation:")
print(rotated_affine)
print("\nWrong (diagonal only):", np.abs(rotated_affine.diagonal()[:3]) * 1000, "μm")
print("Correct (column magnitude):", get_spacing_from_affine(rotated_affine) * 1000, "μm")

Example affine matrix with rotation:
[[-0.01045284  0.          0.09945219 -2.93459034]
 [ 0.         -0.1         0.          8.        ]
 [ 0.09945219  0.          0.01045284 -5.08460236]
 [ 0.          0.          0.          1.        ]]

Wrong (diagonal only): [ 10.45284 100.       10.45284] μm
Correct (column magnitude): [ 99.9999998 100.         99.9999998] μm


## Method 1: Simple Single-Axis Resampling Function

In [4]:
def resample_single_axis(nifti_img, axis='z', new_spacing_mm=0.15, order=1):
    """
    Resample along a single axis of a NIfTI image.
    
    Parameters:
    -----------
    nifti_img : nibabel.Nifti1Image
        Input NIfTI image
    axis : str
        Axis to resample: 'x', 'y', or 'z'
    new_spacing_mm : float
        New spacing in mm for the specified axis
    order : int
        Interpolation order (0=nearest, 1=linear, 3=cubic)
    
    Returns:
    --------
    nibabel.Nifti1Image
        Resampled image
    """
    # Get data and affine
    data = nifti_img.get_fdata()
    affine = nifti_img.affine.copy()
    
    # Get current spacing (handles rotation correctly)
    current_spacing = get_spacing_from_affine(affine)
    
    # Determine axis index
    axis_map = {'x': 0, 'y': 1, 'z': 2}
    axis_idx = axis_map[axis.lower()]
    
    print(f"Original shape: {data.shape}")
    print(f"Original spacing (mm): x={current_spacing[0]:.4f}, y={current_spacing[1]:.4f}, z={current_spacing[2]:.4f}")
    print(f"Original spacing (μm): x={current_spacing[0]*1000:.2f}, y={current_spacing[1]*1000:.2f}, z={current_spacing[2]*1000:.2f}")
    print(f"\nTarget {axis}-spacing: {new_spacing_mm*1000:.2f} μm ({new_spacing_mm:.4f} mm)")
    
    # Calculate zoom factor for specified axis
    zoom_factors = np.ones(3)
    zoom_factors[axis_idx] = current_spacing[axis_idx] / new_spacing_mm
    
    print(f"Zoom factors: x={zoom_factors[0]:.4f}, y={zoom_factors[1]:.4f}, z={zoom_factors[2]:.4f}")
    
    # Resample
    resampled_data = scipy_zoom(data, zoom_factors, order=order, mode='nearest')
    
    print(f"Resampled shape: {resampled_data.shape}")
    print(f"{axis.upper()} dimension: {data.shape[axis_idx]} → {resampled_data.shape[axis_idx]}")
    
    # Update affine matrix (properly handle rotation)
    new_affine = affine.copy()
    col_vector = affine[:3, axis_idx]
    current_magnitude = np.sqrt(np.sum(col_vector ** 2))
    if current_magnitude > 0:
        scale_factor = new_spacing_mm / current_magnitude
        new_affine[:3, axis_idx] = col_vector * scale_factor
    
    # Verify
    verify_spacing = get_spacing_from_affine(new_affine)
    print(f"\nVerified new spacing (μm): x={verify_spacing[0]*1000:.2f}, y={verify_spacing[1]*1000:.2f}, z={verify_spacing[2]*1000:.2f}")
    
    # Create new NIfTI image
    resampled_img = nib.Nifti1Image(resampled_data, new_affine, nifti_img.header)
    
    return resampled_img

## Method 2: Multi-Axis Resampling Function

In [5]:
def resample_nifti(nifti_img, new_spacing=None, new_x=None, new_y=None, new_z=None, order=1):
    """
    Resample NIfTI image with flexible axis control.
    
    Parameters:
    -----------
    nifti_img : nibabel.Nifti1Image
        Input NIfTI image
    new_spacing : tuple, optional
        (x, y, z) spacing in mm. Overrides individual parameters.
    new_x, new_y, new_z : float, optional
        Individual axis spacing in mm. None keeps original.
    order : int
        Interpolation order (0=nearest, 1=linear, 3=cubic)
    
    Returns:
    --------
    nibabel.Nifti1Image
        Resampled image
    """
    # Get data and affine
    data = nifti_img.get_fdata()
    affine = nifti_img.affine.copy()
    
    # Get current spacing
    current_spacing = get_spacing_from_affine(affine)
    
    # Determine target spacing
    if new_spacing is not None:
        target_spacing = np.array(new_spacing, dtype=float)
    else:
        target_spacing = np.array([
            new_x if new_x is not None else current_spacing[0],
            new_y if new_y is not None else current_spacing[1],
            new_z if new_z is not None else current_spacing[2]
        ], dtype=float)
    
    print(f"Original shape: {data.shape}")
    print(f"Original spacing (μm): x={current_spacing[0]*1000:.2f}, y={current_spacing[1]*1000:.2f}, z={current_spacing[2]*1000:.2f}")
    print(f"Target spacing (μm): x={target_spacing[0]*1000:.2f}, y={target_spacing[1]*1000:.2f}, z={target_spacing[2]*1000:.2f}")
    
    # Calculate zoom factors
    zoom_factors = current_spacing / target_spacing
    
    print(f"\nZoom factors: x={zoom_factors[0]:.4f}, y={zoom_factors[1]:.4f}, z={zoom_factors[2]:.4f}")
    for i, (axis, zoom_val) in enumerate(zip(['x', 'y', 'z'], zoom_factors)):
        if zoom_val > 1.0:
            print(f"  {axis}-axis: Downsampling ({zoom_val:.4f}x)")
        elif zoom_val < 1.0:
            print(f"  {axis}-axis: Upsampling ({zoom_val:.4f}x)")
        else:
            print(f"  {axis}-axis: No change")
    
    # Resample
    print("\nResampling...")
    resampled_data = scipy_zoom(data, zoom_factors, order=order, mode='nearest')
    
    print(f"Resampled shape: {resampled_data.shape}")
    for i, axis in enumerate(['X', 'Y', 'Z']):
        change_pct = 100 * (resampled_data.shape[i] / data.shape[i] - 1)
        print(f"  {axis}: {data.shape[i]} → {resampled_data.shape[i]} ({change_pct:+.1f}%)")
    
    # Update affine matrix
    new_affine = affine.copy()
    for i in range(3):
        col_vector = affine[:3, i]
        current_magnitude = np.sqrt(np.sum(col_vector ** 2))
        if current_magnitude > 0:
            scale_factor = target_spacing[i] / current_magnitude
            new_affine[:3, i] = col_vector * scale_factor
    
    # Verify
    verify_spacing = get_spacing_from_affine(new_affine)
    print(f"\nVerified spacing (μm): x={verify_spacing[0]*1000:.2f}, y={verify_spacing[1]*1000:.2f}, z={verify_spacing[2]*1000:.2f}")
    
    # Create new NIfTI image
    resampled_img = nib.Nifti1Image(resampled_data, new_affine, nifti_img.header)
    
    return resampled_img

## Example Usage - Load Your Data

In [ ]:
# Load your NIfTI file
input_path = 'your_mri_volume.nii.gz'  # Replace with your file

# Load the image
nifti_img = nib.load(input_path)

print("Original volume information:")
print(f"Shape: {nifti_img.shape}")
print(f"Affine matrix:\n{nifti_img.affine}")
spacing = get_spacing_from_affine(nifti_img.affine)
print(f"\nVoxel spacing (mm): {spacing}")
print(f"Voxel spacing (μm): {spacing * 1000}")

## Example 1: Resample Z-axis Only

In [ ]:
print("="*60)
print("Example 1: Change Z-spacing to 150 μm")
print("="*60)

resampled_z = resample_single_axis(nifti_img, axis='z', new_spacing_mm=0.15, order=1)

## Example 2: Multi-Axis Resampling

In [ ]:
print("\n" + "="*60)
print("Example 2: Change to 200×100×150 μm")
print("="*60)

resampled_multi = resample_nifti(nifti_img, new_x=0.2, new_z=0.15, order=1)

## Example 3: Using Tuple Input

In [ ]:
print("\n" + "="*60)
print("Example 3: Make isotropic at 150 μm")
print("="*60)

resampled_iso = resample_nifti(nifti_img, new_spacing=(0.15, 0.15, 0.15), order=1)

## Compare Interpolation Methods

In [ ]:
print("\n" + "="*60)
print("Comparing Interpolation Methods (Z to 150 μm)")
print("="*60)

print("\n1. Nearest Neighbor (order=0):")
print("-" * 40)
resampled_nearest = resample_single_axis(nifti_img, 'z', 0.15, order=0)

print("\n2. Linear (order=1) - RECOMMENDED:")
print("-" * 40)
resampled_linear = resample_single_axis(nifti_img, 'z', 0.15, order=1)

print("\n3. Cubic (order=3):")
print("-" * 40)
resampled_cubic = resample_single_axis(nifti_img, 'z', 0.15, order=3)

## Visualization: Compare Slices

In [ ]:
def visualize_slice_comparison(original, nearest, linear, cubic, slice_idx=None):
    """
    Visualize a slice from original and resampled volumes.
    """
    orig_data = original.get_fdata()
    
    if slice_idx is None:
        slice_idx = orig_data.shape[2] // 2
    
    # Calculate corresponding slice in resampled
    resample_slice = int(slice_idx * nearest.shape[2] / orig_data.shape[2])
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    
    axes[0, 0].imshow(orig_data[:, :, slice_idx].T, cmap='gray', origin='lower')
    axes[0, 0].set_title(f'Original\nSlice {slice_idx}/{orig_data.shape[2]}')
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(nearest.get_fdata()[:, :, resample_slice].T, cmap='gray', origin='lower')
    axes[0, 1].set_title(f'Nearest Neighbor\nSlice {resample_slice}/{nearest.shape[2]}')
    axes[0, 1].axis('off')
    
    axes[1, 0].imshow(linear.get_fdata()[:, :, resample_slice].T, cmap='gray', origin='lower')
    axes[1, 0].set_title(f'Linear (Recommended)\nSlice {resample_slice}/{linear.shape[2]}')
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(cubic.get_fdata()[:, :, resample_slice].T, cmap='gray', origin='lower')
    axes[1, 1].set_title(f'Cubic\nSlice {resample_slice}/{cubic.shape[2]}')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

# Uncomment to visualize
# visualize_slice_comparison(nifti_img, resampled_nearest, resampled_linear, resampled_cubic)

## Statistical Comparison

In [ ]:
def compare_statistics(original, resampled, method_name):
    """
    Compare intensity statistics.
    """
    orig_data = original.get_fdata()
    resamp_data = resampled.get_fdata()
    
    print(f"\n{method_name}:")
    print("-" * 50)
    print(f"Original  - Min: {orig_data.min():.2f}, Max: {orig_data.max():.2f}, "
          f"Mean: {orig_data.mean():.2f}, Std: {orig_data.std():.2f}")
    print(f"Resampled - Min: {resamp_data.min():.2f}, Max: {resamp_data.max():.2f}, "
          f"Mean: {resamp_data.mean():.2f}, Std: {resamp_data.std():.2f}")
    print(f"Mean diff: {abs(orig_data.mean() - resamp_data.mean()):.4f}, "
          f"Std diff: {abs(orig_data.std() - resamp_data.std()):.4f}")

compare_statistics(nifti_img, resampled_nearest, "Nearest Neighbor")
compare_statistics(nifti_img, resampled_linear, "Linear")
compare_statistics(nifti_img, resampled_cubic, "Cubic")

## Save Resampled Volume

In [ ]:
# Save the resampled volume (using linear interpolation)
output_path = 'resampled_volume_150um.nii.gz'
nib.save(resampled_linear, output_path)
print(f"Saved resampled volume to: {output_path}")

# Verify the saved file
saved_img = nib.load(output_path)
verify_spacing = get_spacing_from_affine(saved_img.affine)

print(f"\nVerification:")
print(f"Shape: {saved_img.shape}")
print(f"Spacing (mm): {verify_spacing}")
print(f"Spacing (μm): {verify_spacing * 1000}")

## Alternative: Using SimpleITK

SimpleITK handles medical images robustly and is recommended for production use.

In [ ]:
# Uncomment to install
# !pip install SimpleITK --break-system-packages

try:
    import SimpleITK as sitk
    
    def resample_sitk(input_path, output_path, new_spacing=(0.15, 0.1, 0.1), interpolator='linear'):
        """
        Resample using SimpleITK.
        
        Parameters:
        -----------
        input_path : str
            Input NIfTI file
        output_path : str
            Output NIfTI file  
        new_spacing : tuple
            (x, y, z) spacing in mm
        interpolator : str
            'linear', 'nearest', 'bspline'
        """
        # Read image
        image = sitk.ReadImage(input_path)
        
        original_spacing = image.GetSpacing()
        original_size = image.GetSize()
        
        print(f"Original spacing: {original_spacing}")
        print(f"Original size: {original_size}")
        
        # Calculate new size
        new_size = [
            int(round(original_size[i] * (original_spacing[i] / new_spacing[i])))
            for i in range(3)
        ]
        
        print(f"New spacing: {new_spacing}")
        print(f"New size: {new_size}")
        
        # Set interpolator
        interpolator_map = {
            'nearest': sitk.sitkNearestNeighbor,
            'linear': sitk.sitkLinear,
            'bspline': sitk.sitkBSpline
        }
        
        # Resample
        resampler = sitk.ResampleImageFilter()
        resampler.SetOutputSpacing(new_spacing)
        resampler.SetSize(new_size)
        resampler.SetOutputDirection(image.GetDirection())
        resampler.SetOutputOrigin(image.GetOrigin())
        resampler.SetTransform(sitk.Transform())
        resampler.SetDefaultPixelValue(0)
        resampler.SetInterpolator(interpolator_map[interpolator])
        
        resampled = resampler.Execute(image)
        
        # Save
        sitk.WriteImage(resampled, output_path)
        print(f"Saved to: {output_path}")
        
        return resampled
    
    # run
    # resampled_sitk = resample_sitk(
    #     'input.nii.gz',
    #     'output_sitk.nii.gz',
    #     new_spacing=(0.15, 0.1, 0.1),
    #     interpolator='linear'
    # )
    
except ImportError:
    print("SimpleITK not installed. Run: pip install SimpleITK --break-system-packages")

## Summary

**Key Takeaways:**

1. **Spacing Extraction**: Use column vector magnitudes, not diagonal elements
2. **Interpolation**: Linear (order=1) is best for most downsampling
3. **Multi-axis**: You can resample any combination of x, y, z
4. **Rotated Affines**: The code handles them correctly
5. **SimpleITK**: Recommended for production workflows

**Common Use Cases:**
- Reduce file size: Downsample all axes
- Match another volume: Use tuple with target spacing
- Make isotropic: Set all axes to same spacing
- Fix anisotropic data: Resample to uniform spacing